In [8]:
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
from scipy.stats import norm
end = dt.datetime.now()
start = dt.datetime(2022,1,1)

df = yf.download('^NSEI', start, end)
Close = df.Close
Close.tail()


log_returns = np.log(df.Close/df.Close.shift(1)).dropna()

TRADING_DAYS = 20
volatility = log_returns.rolling(window=TRADING_DAYS).std()*np.sqrt(252)


r=0.07 #risk free rate
def round_to_nearest_50(n):  #for finding price of atm option
    return round(n / 50) * 50

T=6/365 #time to expiration

def blackScholes(r, S, K, T, sigma, type="c"):

    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "call":
            price = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "put":
            price = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*norm.cdf(-d1, 0, 1)
        return price
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")


print("Using BSM NIFTY50 Call Option Price is :",round(blackScholes(r, Close['2024-07-12 '], round_to_nearest_50(Close['2024-07-12 ']), T, volatility[ '2024-07-12'], type="call"),2) ,"at",round_to_nearest_50(Close['2024-07-12 ']))
print("Using BSM NIFTY50 Put Option Price is :",round(blackScholes(r, Close['2024-07-12 '], round_to_nearest_50(Close['2024-07-12 ']), T, volatility[ '2024-07-12'], type="put"),2) ,"at",round_to_nearest_50(Close['2024-07-12 ']))


[*********************100%%**********************]  1 of 1 completed

Using BSM NIFTY50 Call Option Price is : 90.58 at 24500
Using BSM NIFTY50 Put Option Price is : 60.25 at 24500


In [17]:
def binomial_tree_call(S, K, r, sigma, T, N):

    dt = T/N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    S_T = S * u**np.arange(N+1) * d**(N-np.arange(N+1))
    V_T = np.maximum(S_T - K, 0)
    for i in range(N-1, -1, -1):
        S_i = S * u**np.arange(i+1) * d**(N-i-np.arange(i+1))
        V_i = np.exp(-r*dt) * (p * V_T[:-1] + (1-p) * V_T[1:])
        V_T = np.maximum(S_i - K, V_i)
    return V_T[0]

print("Using Binomial Pricing NIFTY50 Call Option Price is :",round(binomial_tree_call( Close['2024-07-12 '], round_to_nearest_50(Close['2024-07-12 ']),r,  volatility[ '2024-07-12'], T,4),2) ,"at",round_to_nearest_50(Close['2024-07-12 ']))




Using Binomial Pricing NIFTY50 Call Option Price is : 183.25 at 24500


In [21]:

def simulate_stock_prices(S0, r, sigma, T, N):
    drift = r - 0.5 * sigma ** 2
    std_dev = sigma * np.sqrt(T)
    Z1 = np.random.normal(size=(N,))
    Z2 = np.random.normal(size=(N,))
    ST = S0 * np.exp(drift * T + std_dev * (Z1 + Z2*np.sqrt(1-0.5**2)))
    return ST

stock_prices = simulate_stock_prices( (Close['2024-07-12 ']), r ,volatility[ '2024-07-12'], T, 100000)
payoffs = np.maximum(stock_prices - round_to_nearest_50(Close['2024-07-12 ']), 0)

option_price = np.exp(-r * T) * np.mean(payoffs)

print("The Monte Carlo estimated option price is:", round(option_price,2))



The Monte Carlo estimated option price is: 114.79
